## JSON Scrapes

Example sites:
- <a href="https://a816-health.nyc.gov/ABCEatsRestaurants/#!/Search">Restaurant Inspections</a>
- <a href="https://www1.nyc.gov/site/doh/health/health-topics/school-cafeteria-inspection-results.page">School cafeteria Inspections</a>
- <a href="https://restructuring.primeclerk.com/pge/Home-ClaimInfo">PG&E bankruptcy claims</a>

## Scrape check cashing site

We want all the check cashing locations in New York City.

<a href="http://www.fscny.org/?controller=licensedlocations">Financial services check cashing</a>

## Five-step process
1. On the page or dynamic?
2. Track network data
3. cURL it
4. Convert cURL to Python
5. Python parsing  

In [3]:
## import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from random import randint

In [36]:
## single page trial
cookies = {
    'psi': '0b0645e9-b464-47f9-9db5-c677b9da65a4',
    '_ga': 'GA1.2.2035723594.1663620375',
    'LPVID': 'I1ZjQyMTYzYjhjNDYzZmIz',
    '_gid': 'GA1.2.1125995690.1667925936',
    'LPSID-16393053': 'tZMib4_UR0-dzFCMgsJrAQ',
    '__cf_bm': 'qlAe1mMCKIkTKsky7sBYFvIR7ZXhRzkDEH7Wvcf0PRg-1667925974-0-AUs5TN9b967zwEkSBXcUjo1IqtA3U+zJh8wsIYCDFxfJzQCEtYMaLkDDPJG1smofiGQm4POXAyVyc+l5orjYSKs=',
    'AWSALBTG': 'FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS',
    'AWSALBTGCORS': 'FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS',
    'AWSALB': 'U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v',
    'AWSALBCORS': 'U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v',
}

headers = {
    'authority': 'restructuring.ra.kroll.com',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9,la;q=0.8',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'psi=0b0645e9-b464-47f9-9db5-c677b9da65a4; _ga=GA1.2.2035723594.1663620375; LPVID=I1ZjQyMTYzYjhjNDYzZmIz; _gid=GA1.2.1125995690.1667925936; LPSID-16393053=tZMib4_UR0-dzFCMgsJrAQ; __cf_bm=qlAe1mMCKIkTKsky7sBYFvIR7ZXhRzkDEH7Wvcf0PRg-1667925974-0-AUs5TN9b967zwEkSBXcUjo1IqtA3U+zJh8wsIYCDFxfJzQCEtYMaLkDDPJG1smofiGQm4POXAyVyc+l5orjYSKs=; AWSALBTG=FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS; AWSALBTGCORS=FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS; AWSALB=U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v; AWSALBCORS=U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v',
    'dnt': '1',
    'origin': 'https://restructuring.ra.kroll.com',
    'referer': 'https://restructuring.ra.kroll.com/pge/Home-ClaimInfo',
    'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

data = {
    'ClaimNumber': '',
    'ScheduleNumber': '',
    'CreditorName': '',
    'TotalCurrentClaimAmount': 'Select an Option|Select an Option|',
    'Dates': '|',
    'ScopeValue': 'Claims',
    'QuickSearch': '',
    'Deptors': '0ê4025ê4026ê4093ê',
    'fl': '0',
    '_search': 'false',
    'nd': '1667926242547',
    'rows': '200',
    'page': '2',
    'sidx': 'CreditorName',
    'sord': 'asc',
}

response = requests.post('https://restructuring.ra.kroll.com/pge/Home-LoadClaimData', cookies=cookies, headers=headers, data=data)

In [9]:
# response.text

'{"total":550,"page":1,"records":109900,"rows":[{"ClaimID":1094637,"ScheduleNumber":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eSchedule\\u003c/b\\u003e \\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e \\u003c/span\\u003e","ClaimNumber":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eClaim #\\u003c/b\\u003e\\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e\\u003ca  onclick=ShowClaims(\\u0027MTA5NDYzNw==\\u0027) id=\\u0027claim-form-open-4\\u0027 class=\\u0027link-claim\\u0027 data-toggle=\\u0027modal\\u0027 data-target=\\u0027modal-claim-form\\u0027\\u003e32910\\u003c/a\\u003e\\u003c/span\\u003e","DateFiled":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eFiled Date\\u003c/b\\u003e\\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e10/17/2019\\u003c/span\\u003e","CreditorName":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eCreditor Name\\u003c/b\\u003e\\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e\\u0027OH

In [5]:
# response.content

b'{"total":550,"page":1,"records":109900,"rows":[{"ClaimID":1094637,"ScheduleNumber":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eSchedule\\u003c/b\\u003e \\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e \\u003c/span\\u003e","ClaimNumber":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eClaim #\\u003c/b\\u003e\\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e\\u003ca  onclick=ShowClaims(\\u0027MTA5NDYzNw==\\u0027) id=\\u0027claim-form-open-4\\u0027 class=\\u0027link-claim\\u0027 data-toggle=\\u0027modal\\u0027 data-target=\\u0027modal-claim-form\\u0027\\u003e32910\\u003c/a\\u003e\\u003c/span\\u003e","DateFiled":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eFiled Date\\u003c/b\\u003e\\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e10/17/2019\\u003c/span\\u003e","CreditorName":"\\u003cb class=\\u0027tablesaw-cell-label\\u0027\\u003eCreditor Name\\u003c/b\\u003e\\u003cspan class=\\u0027tablesaw-cell-content\\u0027\\u003e\\u0027O

In [37]:
content = response.json()

In [38]:
content

{'total': 550,
 'page': 2,
 'records': 109900,
 'rows': [{'ClaimID': 1098909,
   'ScheduleNumber': "<b class='tablesaw-cell-label'>Schedule</b> <span class='tablesaw-cell-content'> </span>",
   'ClaimNumber': "<b class='tablesaw-cell-label'>Claim #</b><span class='tablesaw-cell-content'><a  onclick=ShowClaims('MTA5ODkwOQ==') id='claim-form-open-4' class='link-claim' data-toggle='modal' data-target='modal-claim-form'>37177</a></span>",
   'DateFiled': "<b class='tablesaw-cell-label'>Filed Date</b><span class='tablesaw-cell-content'>10/18/2019</span>",
   'CreditorName': "<b class='tablesaw-cell-label'>Creditor Name</b><span class='tablesaw-cell-content'>A. G., minor child</span>",
   'TotalCurrentClaimAmount': "<b class='tablesaw-cell-label'> Claim Value</b><span class='tablesaw-cell-content'> $0.00</span>",
   'DebtorName': "<b class='tablesaw-cell-label'>Debtor Name</b><span class='tablesaw-cell-content'>PG&E Corporation and Pacific G</span>"},
  {'ClaimID': 1098910,
   'ScheduleNumbe

In [13]:
type(content)

dict

In [39]:
target_data = content.get("rows")

In [17]:
target_data

[{'ClaimID': 1094637,
  'ScheduleNumber': "<b class='tablesaw-cell-label'>Schedule</b> <span class='tablesaw-cell-content'> </span>",
  'ClaimNumber': "<b class='tablesaw-cell-label'>Claim #</b><span class='tablesaw-cell-content'><a  onclick=ShowClaims('MTA5NDYzNw==') id='claim-form-open-4' class='link-claim' data-toggle='modal' data-target='modal-claim-form'>32910</a></span>",
  'DateFiled': "<b class='tablesaw-cell-label'>Filed Date</b><span class='tablesaw-cell-content'>10/17/2019</span>",
  'CreditorName': "<b class='tablesaw-cell-label'>Creditor Name</b><span class='tablesaw-cell-content'>'OHANA HEALTH, LLC</span>",
  'TotalCurrentClaimAmount': "<b class='tablesaw-cell-label'> Claim Value</b><span class='tablesaw-cell-content'> $0.00</span>",
  'DebtorName': "<b class='tablesaw-cell-label'>Debtor Name</b><span class='tablesaw-cell-content'>PG&E Corporation and Pacific G</span>"},
 {'ClaimID': 1150032,
  'ScheduleNumber': "<b class='tablesaw-cell-label'>Schedule</b> <span class='ta

In [16]:
type(target_data)

list

In [18]:
len(target_data)

200

In [40]:
df = pd.DataFrame(target_data)
df

,ClaimID,ScheduleNumber,ClaimNumber,DateFiled,CreditorName,TotalCurrentClaimAmount,DebtorName
0,1098909,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
1,1098910,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
2,1099388,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
3,1099863,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
4,1099864,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
...,...,...,...,...,...,...,...
195,1114832,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
196,1049718,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
197,1049719,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
198,1107640,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...


In [41]:
target_cols = list(df.columns)
target_cols

['ClaimID',
 'ScheduleNumber',
 'ClaimNumber',
 'DateFiled',
 'CreditorName',
 'TotalCurrentClaimAmount',
 'DebtorName']

In [42]:
## function to strip_html using BeautifulSoup
def strip_html(target):
    target = str(target)
    try:
        soup = BeautifulSoup(target, "html.parser")
        return (soup.find("span", class_="tablesaw-cell-content").get_text()) ## we're luckly that all columns follow the same pattern
    except:
        return target

In [43]:
## for loop through to strip html
for target in target_cols: 
  df[target] = df[target].apply(lambda x: strip_html(x))
#   print(df1[target])
df

,ClaimID,ScheduleNumber,ClaimNumber,DateFiled,CreditorName,TotalCurrentClaimAmount,DebtorName
0,1098909,,37177,10/18/2019,"A. G., minor child",$0.00,PG&E Corporation and Pacific G
1,1098910,,37178,10/18/2019,"A. G., minor child",$0.00,PG&E Corporation and Pacific G
2,1099388,,37656,10/18/2019,"A. G., minor child",$0.00,PG&E Corporation and Pacific G
3,1099863,,38131,10/18/2019,"A. G., minor child",$0.00,PG&E Corporation and Pacific G
4,1099864,,38132,10/18/2019,"A. G., minor child",$0.00,PG&E Corporation and Pacific G
...,...,...,...,...,...,...,...
195,1114832,,53100,10/19/2019,"A.B., a minor child (EMMANUEL DAVID BROWN, gua...",$0.00,PG&E Corporation and Pacific G
196,1049718,,14367,10/04/2019,"A.B., a minor child (Eridan Bazan, parent)",$0.00,PG&E Corporation and Pacific G
197,1049719,,14368,10/04/2019,"A.B., a minor child (Eridan Bazan, parent)",$0.00,PG&E Corporation and Pacific G
198,1107640,,45908,10/19/2019,"A.B., a minor child (HUMBERTO BOTELLO, guardian)",$0.00,PG&E Corporation and Pacific G


In [49]:
df_all = []
for count in range(1, 5):
    print(f"scraping page {count}")
    cookies = {
        'psi': '0b0645e9-b464-47f9-9db5-c677b9da65a4',
        '_ga': 'GA1.2.2035723594.1663620375',
        'LPVID': 'I1ZjQyMTYzYjhjNDYzZmIz',
        '_gid': 'GA1.2.1125995690.1667925936',
        'LPSID-16393053': 'tZMib4_UR0-dzFCMgsJrAQ',
        '__cf_bm': 'qlAe1mMCKIkTKsky7sBYFvIR7ZXhRzkDEH7Wvcf0PRg-1667925974-0-AUs5TN9b967zwEkSBXcUjo1IqtA3U+zJh8wsIYCDFxfJzQCEtYMaLkDDPJG1smofiGQm4POXAyVyc+l5orjYSKs=',
        'AWSALBTG': 'FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS',
        'AWSALBTGCORS': 'FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS',
        'AWSALB': 'U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v',
        'AWSALBCORS': 'U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v',
    }

    headers = {
        'authority': 'restructuring.ra.kroll.com',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'en-US,en;q=0.9,la;q=0.8',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'psi=0b0645e9-b464-47f9-9db5-c677b9da65a4; _ga=GA1.2.2035723594.1663620375; LPVID=I1ZjQyMTYzYjhjNDYzZmIz; _gid=GA1.2.1125995690.1667925936; LPSID-16393053=tZMib4_UR0-dzFCMgsJrAQ; __cf_bm=qlAe1mMCKIkTKsky7sBYFvIR7ZXhRzkDEH7Wvcf0PRg-1667925974-0-AUs5TN9b967zwEkSBXcUjo1IqtA3U+zJh8wsIYCDFxfJzQCEtYMaLkDDPJG1smofiGQm4POXAyVyc+l5orjYSKs=; AWSALBTG=FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS; AWSALBTGCORS=FXgNTYwTINkCje/wtXjr7Lx9TfwJBKM+vET+aBsBUJR+GDntzenlTY1cSvpZrvvB9aBrVQuu7cwPR5DX6tgfEfrRVZ0g7oIlLgxUr7aVJMwXIgR9D7TiB8o4Osky0uBL2WB9RVfXPLPNjDsz8zAQRgN+3vLdw34Puhv2WF0dOHSS; AWSALB=U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v; AWSALBCORS=U3xLiJ1RcOpK+LjI5TmwhiI051QHgNZaurHNS2nUP+7+CFDdhK66tyIr8Imlyu/kJXF0aGky2bsDEI+qhqse3U+gdVY41raHdTnLR+ePuJjzZcXiYeJtATaWFL7v',
        'dnt': '1',
        'origin': 'https://restructuring.ra.kroll.com',
        'referer': 'https://restructuring.ra.kroll.com/pge/Home-ClaimInfo',
        'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
        'x-requested-with': 'XMLHttpRequest',
    }

    data = {
        'ClaimNumber': '',
        'ScheduleNumber': '',
        'CreditorName': '',
        'TotalCurrentClaimAmount': 'Select an Option|Select an Option|',
        'Dates': '|',
        'ScopeValue': 'Claims',
        'QuickSearch': '',
        'Deptors': '0ê4025ê4026ê4093ê',
        'fl': '0',
        '_search': 'false',
        'nd': '1667926242547',
        'rows': '200',
        'page': count,
        'sidx': 'CreditorName',
        'sord': 'asc',
    }

    response = requests.post('https://restructuring.ra.kroll.com/pge/Home-LoadClaimData', cookies=cookies, headers=headers, data=data)    
    content = response.json()
    target_data = content.get("rows")
    df = pd.DataFrame(target_data)
    df_all.append(df)
    
    snoozer = randint(5,10)
    print(f"snoozing for {snoozer} seconds")
    time.sleep(snoozer)
    
print("done capturing pages")
    

scraping page 1
snoozing for 9 seconds
scraping page 2
snoozing for 9 seconds
scraping page 3
snoozing for 6 seconds
scraping page 4
snoozing for 8 seconds
done capturing pages


In [56]:
## create a function to download each table as CSV
def combine_tables(list_name):
    '''
    takes multiple dataframes in a list, concats them
    export to csv
    para1: list name
    para2: csv file name including .csv as string 
    '''
    df = pd.concat(list_name)
    df.reset_index(inplace = True, drop = True)
#     df.to_csv(csv_name, encoding = "UTF-8", index = False)
    return df

In [50]:
len(df_all)

4

In [57]:
df = combine_tables(df_all)

In [58]:
df

,ClaimID,ScheduleNumber,ClaimNumber,DateFiled,CreditorName,TotalCurrentClaimAmount,DebtorName
0,1094637,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
1,1150032,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
2,1117014,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
3,1130194,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
4,1147535,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
...,...,...,...,...,...,...,...
795,1190078,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
796,1054770,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
797,1093827,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...
798,1112348,<b class='tablesaw-cell-label'>Schedule</b> <s...,<b class='tablesaw-cell-label'>Claim #</b><spa...,<b class='tablesaw-cell-label'>Filed Date</b><...,<b class='tablesaw-cell-label'>Creditor Name</...,<b class='tablesaw-cell-label'> Claim Value</b...,<b class='tablesaw-cell-label'>Debtor Name</b>...


In [59]:
for target in target_cols: 
  df[target] = df[target].apply(lambda x: strip_html(x))
#   print(df1[target])
df

,ClaimID,ScheduleNumber,ClaimNumber,DateFiled,CreditorName,TotalCurrentClaimAmount,DebtorName
0,1094637,,32910,10/17/2019,"'OHANA HEALTH, LLC",$0.00,PG&E Corporation and Pacific G
1,1150032,,84461,10/21/2019,"(Allen) Smith, Julia",$0.00,PG&E Corporation and Pacific G
2,1117014,,55274,10/10/2019,"(Davis) Gerspacher, Abigail N.",$0.00,PG&E Corporation and Pacific G
3,1130194,,64849,10/14/2019,"(Gleeson) Augusto, Maureen T.",$0.00,PG&E Corporation and Pacific G
4,1147535,,82079,10/21/2019,"(Summer Burns and Kelly Burns), R.K.B., a Minor",$0.00,PG&E Corporation and Pacific G
...,...,...,...,...,...,...,...
795,1190078,,97477,03/05/2020,"A.M., a minor child (Rico Martinez, parent)",$0.00,PG&E Corporation and Pacific G
796,1054770,,19370,10/08/2019,"A.M., a minor child (Robert Martin, parent)",$0.00,PG&E Corporation and Pacific G
797,1093827,,32100,10/17/2019,"A.M., a minor child (RoseAnne Pease, parent)",$0.00,PG&E Corporation and Pacific G
798,1112348,,50616,10/19/2019,"A.M., a minor child (Russell Messana, parent)",$0.00,PG&E Corporation and Pacific G


In [60]:
df.to_csv("pge.csv", index = False, encoding = "UTF-8")